# Experimentos de Deep Learning com `nutrition.csv` e Food-101

Este notebook reúne dois experimentos completos e prontos para rodar no Google Colab:

1. **Regressão tabular** usando o arquivo `nutrition.csv` para prever calorias com uma rede totalmente conectada.
2. **Classificação de imagens** com o dataset Food-101 contido em `archive (1)` (subpasta `images/`) utilizando transferência de aprendizado.

Cada bloco inclui comentários e parâmetros configuráveis para facilitar a adaptação ao seu ambiente local ou ao Colab.


## Preparando os dados no Google Colab

1. Faça upload da pasta `archive (1)` (mantendo a estrutura `images/` e `meta/`) e do arquivo `nutrition.csv` para a pasta desejada no Google Drive. Sugestão: `MyDrive/pp4_datasets/`.
2. No Colab, monte o Drive (`drive.mount('/content/drive')`) e aponte `BASE_DATA_DIR` para essa pasta.
3. Ajuste os parâmetros `FOOD_IMAGES_DIR` e `NUTRITION_CSV_PATH` abaixo caso use outro caminho.

> Dica: compacte `archive (1)` em `.zip` antes do upload e descompacte no Colab com `!unzip` para agilizar.


In [5]:
import math
import os
import random
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# ------------------------------------------------------------
# Configurações globais
# ------------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")
    BASE_DATA_DIR = Path("/content/drive/MyDrive/pp4_datasets")
else:
    BASE_DATA_DIR = Path.cwd()

FOOD_IMAGES_DIR = BASE_DATA_DIR / "archive (1)" / "images"
NUTRITION_CSV_PATH = BASE_DATA_DIR / "nutrition.csv"
MODEL_OUTPUT_DIR = BASE_DATA_DIR / "modelos_treinados"
MODEL_OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Rodando no Colab? {IN_COLAB}")
print(f"Base de dados: {BASE_DATA_DIR}")
print(f"Imagens Food-101: {FOOD_IMAGES_DIR.exists()}")
print(f"Tabela de nutrição: {NUTRITION_CSV_PATH.exists()}")

AUTOTUNE = tf.data.AUTOTUNE
gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(f"GPUs detectadas: {len(gpus)}")


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
if FOOD_IMAGES_DIR.exists():
    classes = sorted([d.name for d in FOOD_IMAGES_DIR.iterdir() if d.is_dir()])
    sample_counts = {c: len(list((FOOD_IMAGES_DIR / c).glob("*.jpg"))) for c in classes[:5]}
    print(f"Total de classes detectadas: {len(classes)} (mostrando 5)")
    print(sample_counts)
else:
    print("⚠️ Pasta de imagens não localizada. Ajuste FOOD_IMAGES_DIR acima.")

if NUTRITION_CSV_PATH.exists():
    preview = pd.read_csv(NUTRITION_CSV_PATH, nrows=5)
    display(preview.head())
else:
    print("⚠️ Arquivo nutrition.csv não encontrado. Ajuste NUTRITION_CSV_PATH.")


## Experimento 1 — Regressão de calorias com `nutrition.csv`

Objetivo: prever o valor calórico (kcal) com base nos demais nutrientes usando uma rede densa. Este processo inclui limpeza automática (remoção de unidades como `g`, `mg`, etc.), normalização e validação cruzada simples.


In [ ]:
def load_and_prepare_nutrition(csv_path: Path, drop_threshold: float = 0.4):
    assert csv_path.exists(), f"Arquivo {csv_path} não encontrado."
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()

    # Remove colunas textuais que não contribuem para o modelo
    text_cols = ["name", "serving_size"]
    existing_text_cols = [c for c in text_cols if c in df.columns]
    df = df.drop(columns=existing_text_cols)

    def _to_numeric(series: pd.Series) -> pd.Series:
        cleaned = series.astype(str).str.replace(r"[^0-9eE\.\-]", "", regex=True)
        cleaned = cleaned.replace({"": np.nan, "nan": np.nan})
        return pd.to_numeric(cleaned, errors="coerce")

    df = df.apply(_to_numeric)

    # Descarta colunas com muitos NaNs
    null_ratio = df.isna().mean()
    cols_to_drop = null_ratio[null_ratio > drop_threshold].index.tolist()
    df = df.drop(columns=cols_to_drop)

    # Remove linhas quase vazias e preenche o restante com mediana
    min_non_na = int(df.shape[1] * 0.6)
    df = df.dropna(thresh=min_non_na)
    df = df.fillna(df.median(numeric_only=True))

    target = "calories"
    if target not in df.columns:
        raise ValueError("Coluna 'calories' não encontrada após limpeza.")

    feature_df = df.drop(columns=[target])
    scaler = StandardScaler()
    features = scaler.fit_transform(feature_df)
    targets = df[target].values.astype(np.float32)

    feature_names = feature_df.columns.tolist()
    return features, targets, feature_names, scaler

X, y, nutrition_feature_names, nutrition_scaler = load_and_prepare_nutrition(NUTRITION_CSV_PATH)
print(f"Shape final: X={X.shape}, y={y.shape}, nº de features={len(nutrition_feature_names)}")


In [ ]:
test_size = 0.2
batch_size = 64

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=test_size, random_state=SEED
)

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(buffer_size=len(X_train), seed=SEED)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val))
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

len_train = len(X_train)
len_val = len(X_val)
print(f"Treino: {len_train} amostras | Validação: {len_val} amostras")


In [ ]:
def build_tabular_model(input_dim: int) -> keras.Model:
    inputs = keras.Input(shape=(input_dim,), name="nutrition_features")
    x = layers.Dense(256, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, name="calories_output")(x)
    return keras.Model(inputs, outputs, name="nutrition_regressor")

nutrition_model = build_tabular_model(X.shape[1])
nutrition_model.summary()

nutrition_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=[
        keras.metrics.RootMeanSquaredError(name="rmse"),
        keras.metrics.MeanAbsoluteError(name="mae"),
    ],
)

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_rmse", patience=10, restore_best_weights=True
)
plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_rmse", factor=0.5, patience=5, min_lr=1e-5
)

history = nutrition_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stop, plateau],
    verbose=2,
)



In [ ]:
import pandas as pd

history_df = pd.DataFrame(history.history)
display(history_df.tail())

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
history_df[["rmse", "val_rmse"]].plot(ax=axes[0])
axes[0].set_title("RMSE vs Épocas")
axes[0].set_xlabel("épocas")
axes[0].set_ylabel("rmse")
axes[0].grid(True, linestyle="--", alpha=0.3)

history_df[["mae", "val_mae"]].plot(ax=axes[1])
axes[1].set_title("MAE vs Épocas")
axes[1].set_xlabel("épocas")
axes[1].set_ylabel("mae")
axes[1].grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()


### Avaliação rápida do regressor

O bloco abaixo calcula métricas no conjunto de validação, gera previsões de exemplo e salva o modelo/normalizador para reutilização no Colab ou em produção.


In [ ]:
from sklearn.metrics import r2_score

val_preds = nutrition_model.predict(val_ds).squeeze()
val_true = y_val
rmse = tf.keras.metrics.RootMeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError()
rmse.update_state(val_true, val_preds)
mae.update_state(val_true, val_preds)
print({"rmse": float(rmse.result()), "mae": float(mae.result())})
print({"r2": r2_score(val_true, val_preds)})

sample_idx = np.random.choice(len(X_val), size=5, replace=False)
sample_features = X_val[sample_idx]
sample_truth = y_val[sample_idx]
sample_preds = nutrition_model.predict(sample_features).squeeze()

comparison_df = pd.DataFrame(
    {
        "calories_real": sample_truth,
        "calories_previsto": sample_preds,
    }
)
display(comparison_df)

regressor_path = MODEL_OUTPUT_DIR / "nutrition_regressor.keras"
nutrition_model.save(regressor_path)

scaler_path = MODEL_OUTPUT_DIR / "nutrition_scaler.npy"
np.save(scaler_path, {
    "mean": nutrition_scaler.mean_,
    "scale": nutrition_scaler.scale_,
    "feature_names": np.array(nutrition_feature_names),
})

print(f"Modelo salvo em {regressor_path}")
print(f"Scaler salvo em {scaler_path}")


## Experimento 2 — Classificação de imagens com Food-101

Objetivo: treinar um classificador de 101 classes utilizando transferência de aprendizado (EfficientNetB0). Habilite GPU no Colab para tempos de treino aceitáveis. É possível limitar a fração de dados para prototipagem rápida ajustando `TRAIN_SPLIT_FRACTION`.


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_SPLIT_FRACTION = 0.2  # use <=1.0 para reduzir a quantidade de imagens de treino
VAL_SPLIT = 0.1
MAX_EPOCHS = 25
LEARNING_RATE = 2e-4

print({
    "IMG_SIZE": IMG_SIZE,
    "BATCH_SIZE": BATCH_SIZE,
    "TRAIN_SPLIT_FRACTION": TRAIN_SPLIT_FRACTION,
    "VAL_SPLIT": VAL_SPLIT,
    "MAX_EPOCHS": MAX_EPOCHS,
})


In [ ]:
def build_food_datasets(
    images_dir: Path,
    image_size: tuple[int, int] = IMG_SIZE,
    batch_size: int = BATCH_SIZE,
    val_split: float = VAL_SPLIT,
    subset_fraction: float = TRAIN_SPLIT_FRACTION,
):
    assert images_dir.exists(), f"Pasta {images_dir} não encontrada."
    assert 0.0 < val_split < 1.0, "VAL_SPLIT deve estar entre 0 e 1"
    assert 0.0 < subset_fraction <= 1.0, "TRAIN_SPLIT_FRACTION deve estar entre 0 e 1"

    train_ds = tf.keras.utils.image_dataset_from_directory(
        images_dir,
        validation_split=val_split,
        subset="training",
        seed=SEED,
        image_size=image_size,
        batch_size=batch_size,
        label_mode="categorical",
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        images_dir,
        validation_split=val_split,
        subset="validation",
        seed=SEED,
        image_size=image_size,
        batch_size=batch_size,
        label_mode="categorical",
    )

    if subset_fraction < 1.0:
        train_batches = tf.data.experimental.cardinality(train_ds).numpy()
        keep_batches = max(1, int(train_batches * subset_fraction))
        train_ds = train_ds.take(keep_batches)
        print(f"Subamostrando treino para {keep_batches} lotes")

    class_names = train_ds.class_names

    def configure(ds: tf.data.Dataset, shuffle: bool = False) -> tf.data.Dataset:
        if shuffle:
            ds = ds.shuffle(1024, seed=SEED)
        return ds.prefetch(AUTOTUNE)

    return configure(train_ds, shuffle=True), configure(val_ds), class_names


food_train_ds, food_val_ds, food_class_names = build_food_datasets(FOOD_IMAGES_DIR)
num_classes = len(food_class_names)
print(f"Classes: {num_classes}")



In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.1),
    ],
    name="augmentation_block",
)

preprocess_input = tf.keras.applications.efficientnet.preprocess_input



In [ ]:
def build_food_model(num_classes: int) -> keras.Model:
    inputs = keras.Input(shape=(*IMG_SIZE, 3), name="food_image")
    x = data_augmentation(inputs)
    x = preprocess_input(x)

    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_tensor=x,
    )
    base_model.trainable = False

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="food_logits")(x)

    model = keras.Model(inputs, outputs, name="food101_classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


food_model = build_food_model(num_classes)
food_model.summary()



In [ ]:
food_ckpt_path = MODEL_OUTPUT_DIR / "food101_classifier.keras"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=food_ckpt_path,
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1,
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        restore_best_weights=True,
    ),
]

food_history = food_model.fit(
    food_train_ds,
    validation_data=food_val_ds,
    epochs=MAX_EPOCHS,
    callbacks=callbacks,
    verbose=2,
)



In [ ]:
food_history_df = pd.DataFrame(food_history.history)
display(food_history_df.tail())

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
food_history_df[["accuracy", "val_accuracy"]].plot(ax=axes[0])
axes[0].set_title("Acurácia Food-101")
axes[0].set_xlabel("épocas")
axes[0].set_ylabel("acurácia")
axes[0].grid(True, linestyle="--", alpha=0.3)

food_history_df[["loss", "val_loss"]].plot(ax=axes[1])
axes[1].set_title("Loss Food-101")
axes[1].set_xlabel("épocas")
axes[1].set_ylabel("loss")
axes[1].grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()



In [ ]:
val_metrics = food_model.evaluate(food_val_ds, return_dict=True, verbose=0)
print(val_metrics)

val_images, val_labels = next(iter(food_val_ds.take(1)))
val_probs = food_model.predict(val_images)
val_preds = tf.argmax(val_probs, axis=1)
val_true = tf.argmax(val_labels, axis=1)

for idx in range(min(5, val_images.shape[0])):
    true_label = food_class_names[int(val_true[idx])]
    pred_label = food_class_names[int(val_preds[idx])]
    confidence = tf.reduce_max(val_probs[idx]).numpy()
    print(f"Imagem {idx}: real={true_label} | previsto={pred_label} ({confidence:.2%})")

plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(val_images[i].numpy().astype("uint8"))
    plt.title(f"{food_class_names[int(val_preds[i])]}\nreal: {food_class_names[int(val_true[i])]}")
    plt.axis("off")
plt.tight_layout()
plt.show()



### Fine-tuning opcional

Descomente o bloco abaixo caso queira liberar as últimas camadas do EfficientNet para ajuste fino depois que o cabeçalho estiver estável. Recomendo reduzir bastante a `learning_rate` antes de rodar.



In [ ]:
# base_model = food_model.get_layer("efficientnetb0")
# for layer in base_model.layers[-30:]:
#     layer.trainable = True
# food_model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE * 0.1),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )
# fine_tune_history = food_model.fit(
#     food_train_ds,
#     validation_data=food_val_ds,
#     epochs=5,
#     verbose=2,
# )



In [ ]:
np.save(MODEL_OUTPUT_DIR / "food101_class_names.npy", np.array(food_class_names))
print(f"Classes salvas em {MODEL_OUTPUT_DIR / 'food101_class_names.npy'}")



## Como levar tudo para o Google Colab

1. Comprima a pasta `archive (1)` em `.zip` e faça upload para o Drive. No Colab, use `!unzip` dentro de `/content/drive/MyDrive/pp4_datasets/`.
2. Suba `nutrition.csv` para a mesma pasta.
3. Abra este notebook no Colab, monte o Drive e ajuste `BASE_DATA_DIR` se usar outro caminho.
4. Execute todas as células na ordem. Os modelos e artefatos serão salvos em `BASE_DATA_DIR / modelos_treinados`, facilitando o download posterior (`files.download`).

